In [ ]:
from src.main.main import load_data

x, y, classes = load_data()

In [ ]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

n_splits = 1
if y.ndim > 1:
    y_labels = np.argmax(y, axis=1)
else:
    y_labels = y

skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
histories = []

for fold_no, (train_index, test_index) in enumerate(skf.split(x, y_labels), start=1):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y_labels[train_index], y_labels[test_index]
    y_tr, y_te = y[train_index], y[test_index]